# Phân tích xu hướng mua sắm (Shopping Trend Analysis)

In [ ]:
# Import các thư viện cần thiết
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.cluster import KMeans
from mlxtend.frequent_patterns import apriori, association_rules


In [ ]:
# Đọc dữ liệu từ file CSV (đặt file CSV trong cùng thư mục với Notebook)
data = pd.read_csv('shopping_trends.csv')  # Thay 'shopping_trends.csv' bằng tên file thực tế
print('Kích thước dữ liệu:', data.shape)
print(data.head())
print(data.info())


In [ ]:
# Kiểm tra dữ liệu thiếu và trùng lặp
print('Số lượng giá trị thiếu:', data.isnull().sum().sum())
print('Số lượng dòng trùng lặp:', data.duplicated().sum())


In [ ]:
categorical_cols = ['Gender', 'Item Purchased', 'Category', 'Size', 'Color', 'Season']
numerical_cols = ['Age', 'Purchase Amount (USD)', 'Review Rating', 'Previous Purchases']


In [ ]:
plt.figure(figsize=(6,6))
data['Gender'].value_counts().plot.pie(autopct='%1.1f%%', colors=['blue','red'])
plt.title('Phân bố giới tính khách hàng')
plt.show()


In [ ]:
plt.figure(figsize=(14,6))
sns.countplot(data=data, x='Item Purchased', hue='Gender')
plt.title('Số lượng đơn mua theo giới tính')
plt.xticks(rotation=45)
plt.show()


In [ ]:
plt.figure(figsize=(8,6))
correlation_matrix = data[numerical_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Ma trận tương quan các biến số')
plt.show()


In [ ]:
cluster_features = data[['Age', 'Purchase Amount (USD)', 'Previous Purchases']]
scaler = StandardScaler()
scaled_features = scaler.fit_transform(cluster_features)

kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(scaled_features)
data['Cluster'] = kmeans.labels_

print(data[['Age', 'Purchase Amount (USD)', 'Previous Purchases', 'Cluster']].head())


In [ ]:
plt.figure(figsize=(8,6))
plt.scatter(scaled_features[:,0], scaled_features[:,1], c=data['Cluster'], cmap='viridis')
plt.title('Phân cụm khách hàng (K-Means)')
plt.xlabel('Age (scaled)')
plt.ylabel('Purchase Amount (scaled)')
plt.show()


In [ ]:
X = data[['Age', 'Previous Purchases', 'Review Rating']]
y = data['Purchase Amount (USD)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_test)
print('Decision Tree MAE:', mean_absolute_error(y_test, preds))
print('Decision Tree RMSE:', mean_squared_error(y_test, preds, squared=False))


In [ ]:
basket = data.groupby(['Customer ID', 'Item Purchased'])['Item Purchased'].count().unstack().fillna(0)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1.0)

print('Top 5 luật kết hợp:')
print(rules.sort_values('lift', ascending=False).head())
